# Problem

Design a data structure that follows the constraints of a Least Recently Used (LRU) cache.

Implement the LRUCache class:

- LRUCache(int capacity) Initialize the LRU cache with positive size capacity.
- int get(int key) Return the value of the key if the key exists, otherwise return -1.
- void put(int key, int value) Update the value of the key if the key exists. Otherwise, add the key-value pair to the cache. If the number of keys exceeds the capacity from this operation, evict the least recently used key.
- The functions get and put must each run in O(1) average time complexity.



# Thought Process

LRU - used the earliest time, putting and getting the keys in cache is equal to using them

Use hash map that stores key and double link list node

Double link list node will have head and tail - From head to tail would be most recently used to least recently used

Head will point to tail in right direction (next), tail will point to head in left direction (prev)

Put function would check the key, if not exist -> input the key,value pair right after head, if exist, delete the original address of the key and put it right after the head 

If get function is called, if key exists-> return the value and delete the original address of the key and put it right after the head

In [2]:
class LRUCache:
    # Nested class showing DLL contains, key, value and pointers to prev and next nodes
    class Node:
        def __init__(self, key, val):
            self.key = key
            self.val = val
            self.prev = None
            self.next = None

    # Dummy head and tail nodes, pointing each other and a hash map
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.head = self.Node(-1, -1) # Initialize head (-1,-1) for sentinel values
        self.tail = self.Node(-1, -1) # Initialize tail (-1,-1) for sentinel values
        self.head.next = self.tail 
        self.tail.prev = self.head
        self.hash_map = {}
    
    def addNode(self, newNode):
        temp = self.head.next # temporary node and put it right after head node
        newNode.next = temp # new node's next node would be temporary node, putting in front of first real node
        newNode.prev = self.head # new node's prev node would be head node
        self.head.next = newNode # head node's next node would be new node
        temp.prev = newNode # temp node's prev node would be new node

    def deleteNode(self, delNode):
        prev_del = delNode.prev # prev node before the deleted node
        next_del = delNode.next # next node after the deleted node
        prev_del.next = next_del # now point prev node and next node each other
        next_del.prev = prev_del

    def get(self, key: int) -> int:
        if key in self.hash_map:
            resultNode = self.hash_map[key] # get the node using the key from hash map
            answer = resultNode.val # get the value from the node
            del self.hash_map[key] # delete the key from hash map
            self.deleteNode(resultNode) # delete the node from DLL original positon
            self.addNode(resultNode) # add the node right after the head node
            self.hash_map[key] = self.head.next # point the key right after head node
            return answer
        return -1 # if key does not exist

    def put(self, key: int, value: int) -> None:
        if key in self.hash_map: 
            curr = self.hash_map[key]
            del self.hash_map[key] # delete the key from hashmap
            self.deleteNode(curr) # delete the node from DLL

        if len(self.hash_map) == self.capacity: # if capacity is full
            del self.hash_map[self.tail.prev.key] # delete the least recently used key which is in front of the tail node
            self.deleteNode(self.tail.prev) # delete the least recently used node
        
        self.addNode(self.Node(key,value)) # add the node to DLL
        self.hash_map[key] = self.head.next # point it right after head
        

In [3]:
def test_LRUCache():
    # Test case 1: Basic put and get
    cache = LRUCache(2)
    cache.put(1, 1)
    cache.put(2, 2)
    assert cache.get(1) == 1, "get(1) should return 1"
    cache.put(3, 3) # This should evict key 2
    assert cache.get(2) == -1, "get(2) should return -1 because it was evicted"

    # Test case 2: Replacing value for existing key
    cache.put(1, 10)
    assert cache.get(1) == 10, "get(1) should return 10 after updating its value"

    # Test case 3: LRU property check
    cache.put(4, 4) # This should evict key 3
    assert cache.get(3) == -1, "get(3) should return -1 because it was evicted"
    assert cache.get(4) == 4, "get(4) should return 4"
    assert cache.get(1) == 10, "get(1) should return 10"

    # Test case 4: Capacity constraint
    cache = LRUCache(1)
    cache.put(1, 1)
    cache.put(2, 2) # This should evict key 1
    assert cache.get(1) == -1, "get(1) should return -1 because it was evicted"
    assert cache.get(2) == 2, "get(2) should return 2"

    print("All tests passed successfully.")

test_LRUCache()


All tests passed successfully.
